In [ ]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn import metrics
import statsmodels.api as sm
%matplotlib inline

from statsmodels.stats.outliers_influence import variance_inflation_factor 



In [ ]:
train=pd.read_csv("../input/train.csv")
test=pd.read_csv("../input/test.csv")

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
train.info()

In [ ]:
#check for NULL values in the data.
train.isna().sum()

In [ ]:
#lets extract the different date related features from the datetime object
train['hour']=[t.hour for t in pd.DatetimeIndex(train.datetime)]
train['day']=[t.dayofweek for t in pd.DatetimeIndex(train.datetime)]
train['month']=[t.month for t in pd.DatetimeIndex(train.datetime)]
train['year']=[t.year for t in pd.DatetimeIndex(train.datetime)]
train['quarter']=[t.quarter for t in pd.DatetimeIndex(train.datetime)]

In [ ]:
#lets extract the different date related features from the datetime object
test['hour']=[t.hour for t in pd.DatetimeIndex(test.datetime)]
test['day']=[t.dayofweek for t in pd.DatetimeIndex(test.datetime)]
test['month']=[t.month for t in pd.DatetimeIndex(test.datetime)]
test['year']=[t.year for t in pd.DatetimeIndex(test.datetime)]
test['quarter']=[t.quarter for t in pd.DatetimeIndex(test.datetime)]

In [ ]:
#drop the field datetime
train.drop(['datetime'],inplace=True,axis=1)
test.drop(['datetime'],inplace=True,axis=1)

In [ ]:
#lets use the forward selection of variable approach.
#create models separately for registered and casual users as the parameters impact these two measures separately.

In [ ]:
#lets start with a model that can predict the registered users mode accurately.

In [ ]:
# set some hypothesis based on our initial understanding.we will validate them during the exploratory data analysis

In [ ]:
#H1 : Registered users probably use the bikes more during the office hours compared to non office hours.
#H2 : Registered users count on the weekends will be lower compared to the weekdays and holidays
#H3 : Registered users count will increase as the time increases.
#H4 : people tend to use less bikes during high humidity
#H5 : demand will be more during the good climate
#H6 : demand will be low during high temperature
#H7 : low demand during the wind speed is high.

In [ ]:
#lets confirm whether the hypothesis we have defined are correct by doing EDA.
#every hypotheis is like a story.we are going to add story to our model in terms of the parameters.

In [ ]:
train.head()

In [ ]:
#average demand for the registered and casual users
f,axis=plt.subplots(1,2,figsize=(15,6))
b1=sns.barplot(data=train,x='hour',y='registered',ax=axis[0])
b2=sns.barplot(data=train,x='hour',y='casual',ax=axis[1])

In [ ]:
#EDA support the H1.lets use this variable in the model.

In [ ]:
train_x1=train['hour']
train_R_y1=train['registered']
train_C_y1=train['casual']

In [ ]:
#model to predict the registed users count
model_R_1=sm.OLS(train_R_y1,train_x1).fit()
print(model_R_1.summary())

In [ ]:
#model to predict the casual users count
model_C_1=sm.OLS(train_C_y1,train_x1).fit()
print(model_C_1.summary())

In [ ]:
#predict the Registered and casual users
predict_R_1=model_R_1.predict(train_x1)
predict_C_1=model_C_1.predict(train_x1)

In [ ]:
predict_count_1=predict_R_1+predict_C_1

In [ ]:
#visualize predict_count_1 and actual count
plt.Figure(figsize=(5,5))
plt.scatter(predict_count_1,train['count'])
plt.plot([0,1000],[0,1000],color='red')
plt.xlim=(-100,1000)
plt.ylim=(-100,1000)
plt.xlabel('prediction')
plt.ylabel('train_y')
plt.title('Linear Regression Model')

In [ ]:
#define RMSLE
def rmsle(prediction,actual):
    log1=np.array([np.log(v+1) for v in prediction])
    log2=np.array([np.log(v+1) for v in actual])
    calc=(log1-log2)**2
    return np.sqrt(np.mean(calc))

In [ ]:
rmsle(predict_count_1,train['count'])

In [ ]:
# above is the approach I am takin for this project

In [ ]:
# now lets consider the #registered users only and add more features and improve the R squared.

In [ ]:
#H2 : Registered users count on the weekends will be lower compared to the weekdays and holidays

sns.barplot(data=train,x='day',y='registered')


In [ ]:

sns.barplot(data=train,x='workingday',y='registered')

In [ ]:
# EDA proves our hypothesis. On working days the demand is more for registered users.

In [ ]:
train_x1=train[['hour','workingday']]
train_R_y1=train['registered']

In [ ]:
#model to predict the registed users count
model_R_2=sm.OLS(train_R_y1,train_x1).fit()
print(model_R_2.summary())

In [ ]:
#H3 : Registered users count will increase as the time increases.

#lets create a field for identifying the quarter
train['Quarter_number']=np.where((train['year']==2011) & (train['month']<=3),1,
                  np.where((train['year']==2011) & (train['month']>3) & (train['month']<=6),2,
                         np.where((train['year']==2011) & (train['month']>6) & (train['month']<=9),3,
                                np.where((train['year']==2011) & (train['month']>9) & (train['month']<=12),4,
                                       np.where((train['year']==2012) & (train['month']<3) ,5,
                                              np.where((train['year']==2012) & (train['month']>3) & (train['month']<=6),6,
                                                     np.where((train['year']==2012) & (train['month']>6) & (train['month']<=9),7,
                                                            np.where((train['year']==2012) & (train['month']>9) & (train['month']<=12),8,0
                                                                    ))))))))


In [ ]:


#lets create a field for identifying the quarter in the test data set
test['Quarter_number']=np.where((test['year']==2011) & (test['month']<=3),1,
                  np.where((test['year']==2011) & (test['month']>3) & (test['month']<=6),2,
                         np.where((test['year']==2011) & (test['month']>6) & (test['month']<=9),3,
                                np.where((test['year']==2011) & (test['month']>9) & (test['month']<=12),4,
                                       np.where((test['year']==2012) & (test['month']<3) ,5,
                                              np.where((test['year']==2012) & (test['month']>3) & (test['month']<=6),6,
                                                     np.where((test['year']==2012) & (test['month']>6) & (test['month']<=9),7,
                                                            np.where((test['year']==2012) & (test['month']>9) & (test['month']<=12),8,0
                                                                    ))))))))


In [ ]:
train_x1=train[['hour','workingday','Quarter_number']]
train_R_y1=train['registered']

In [ ]:
train.Quarter_number.unique()

In [ ]:
#model to predict the registed users count
model_R_2=sm.OLS(train_R_y1,train_x1).fit()
print(model_R_2.summary())

In [ ]:
#The R squared value has increased to .627.

In [ ]:
#H6 : demand will be low during high temperature
#lets do an EDA to check how temperature affect the demand

plt.scatter(x='humidity',y='registered',data=train)

In [ ]:
#lets create buckets for humidity
train['humiditybins']=np.floor(train['humidity'])//5
train['tempbins']=np.floor(train['temp'])//5
train['atempbins']=np.floor(train['atemp'])//5
train['windspeedbins']=np.floor(train['windspeed'])//5


In [ ]:
#lets create buckets for humidity
test['humiditybins']=np.floor(test['humidity'])//5
test['tempbins']=np.floor(test['temp'])//5
test['atempbins']=np.floor(test['atemp'])//5
test['windspeedbins']=np.floor(test['windspeed'])//5


In [ ]:
sns.barplot(data=train,x='humiditybins',y='registered')

In [ ]:
sns.barplot(data=train,x='windspeedbins',y='registered')

In [ ]:
sns.barplot(data=train,x='tempbins',y='registered')

In [ ]:
sns.barplot(data=train,x='atempbins',y='registered')

In [ ]:
train_x1=train[['hour','workingday','Quarter_number','humiditybins','tempbins','atempbins','windspeedbins']]
train_R_y1=train['registered']

In [ ]:
#model to predict the registed users count
model_R_2=sm.OLS(train_R_y1,train_x1).fit()
print(model_R_2.summary())

In [ ]:
#lets check the VIF value to see if any multicollinearity
vif=pd.DataFrame()
vif['viffactor']=[variance_inflation_factor(train_x1.values,i) for i in range(train_x1.shape[1])]
vif['features']=train_x1.columns
vif

In [ ]:
#looks like the vif factor for atempbins is high.lets remove and check the model

In [ ]:
train_x1=train[['hour','workingday','Quarter_number','humiditybins','tempbins','windspeedbins']]
train_R_y1=train['registered']

In [ ]:
#model to predict the registed users count
model_R_2=sm.OLS(train_R_y1,train_x1).fit()
print(model_R_2.summary())

In [ ]:
vif=pd.DataFrame()
vif['viffactor']=[variance_inflation_factor(train_x1.values,i) for i in range(train_x1.shape[1])]
vif['features']=train_x1.columns
vif

In [ ]:
sns.barplot(data=train,x='weather',y='registered')

In [ ]:
#lets create dummy variables for weather
weather_1=pd.get_dummies(train['weather'],prefix='weather')
train=pd.concat([train,weather_1],axis=1)
train.head()

In [ ]:
#lets create dummy variables for weather
weather_1=pd.get_dummies(test['weather'],prefix='weather')
test=pd.concat([test,weather_1],axis=1)
test.head()

In [ ]:
train_x1=train[['hour','workingday','Quarter_number','humiditybins','tempbins','windspeedbins','weather_1','weather_2','weather_3','weather_4']]
train_R_y1=train['registered']

In [ ]:
#model to predict the registed users count
model_R_2=sm.OLS(train_R_y1,train_x1).fit()
print(model_R_2.summary())

In [ ]:
#H5 : demand will be more during the good climate


In [ ]:
vif=pd.DataFrame()
vif['viffactor']=[variance_inflation_factor(train_x1.values,i) for i in range(train_x1.shape[1])]
vif['features']=train_x1.columns
vif

In [ ]:
#remove weather_4
train_x1=train[['hour','workingday','Quarter_number','humiditybins','tempbins','windspeedbins','weather_1','weather_2','weather_3']]
train_R_y1=train['registered']


In [ ]:
#model to predict the registed users count
model_R_2=sm.OLS(train_R_y1,train_x1).fit()
print(model_R_2.summary())


In [ ]:
vif=pd.DataFrame()
vif['viffactor']=[variance_inflation_factor(train_x1.values,i) for i in range(train_x1.shape[1])]
vif['features']=train_x1.columns
vif

In [ ]:
#remove weather_2
train_x1=train[['hour','workingday','Quarter_number','humiditybins','tempbins','windspeedbins','weather_1','weather_3']]
train_R_y1=train['registered']
#model to predict the registed users count
model_R_2=sm.OLS(train_R_y1,train_x1).fit()
print(model_R_2.summary())

In [ ]:
vif=pd.DataFrame()
vif['viffactor']=[variance_inflation_factor(train_x1.values,i) for i in range(train_x1.shape[1])]
vif['features']=train_x1.columns
vif

In [ ]:
#lets create dummy variables for season
season_1=pd.get_dummies(train['season'],prefix='season')
train=pd.concat([train,season_1],axis=1)
train.head()

In [ ]:
#lets create dummy variables for season
season_1=pd.get_dummies(test['season'],prefix='season')
test=pd.concat([test,season_1],axis=1)
test.head()

In [ ]:

train_x1=train[['hour','workingday','Quarter_number','humiditybins','tempbins','windspeedbins','weather_1','weather_3','season_1','season_2','season_3','season_4']]
train_R_y1=train['registered']
#model to predict the registed users count
model_R_2=sm.OLS(train_R_y1,train_x1).fit()
print(model_R_2.summary())

In [ ]:
vif=pd.DataFrame()
vif['viffactor']=[variance_inflation_factor(train_x1.values,i) for i in range(train_x1.shape[1])]
vif['features']=train_x1.columns
vif

In [ ]:

train_x1=train[['hour','workingday','Quarter_number','humiditybins','tempbins','windspeedbins','weather_1','weather_3','season_1','season_2','season_4']]
train_R_y1=train['registered']
#model to predict the registed users count
model_R_2=sm.OLS(train_R_y1,train_x1).fit()
print(model_R_2.summary())

In [ ]:
vif=pd.DataFrame()
vif['viffactor']=[variance_inflation_factor(train_x1.values,i) for i in range(train_x1.shape[1])]
vif['features']=train_x1.columns
vif

In [ ]:

train_x1=train[['hour','workingday','Quarter_number','humiditybins','tempbins','weather_1','weather_3','season_1','season_2','season_4']]
train_R_y1=train['registered']
#model to predict the registed users count
model_R_2=sm.OLS(train_R_y1,train_x1).fit()
print(model_R_2.summary())

In [ ]:

test_x1=test[['hour','workingday','Quarter_number','humiditybins','tempbins','weather_1','weather_3','season_1','season_2','season_4']]



In [ ]:
#predict the Registered and casual users
predict_R_2=model_R_2.predict(train_x1)

#visualize predict_count_1 and actual count
plt.Figure(figsize=(5,5))
plt.scatter(predict_R_2,train['registered'])
plt.plot([0,1000],[0,1000],color='red')
plt.xlim=(-100,1000)
plt.ylim=(-100,1000)
plt.xlabel('prediction')
plt.ylabel('train_y')
plt.title('Linear Regression Model')

In [ ]:
#predict the Registered and casual users
predict_R_2_test=model_R_2.predict(test_x1)



In [ ]:
predict_R_2=np.where(predict_R_2<0,10,predict_R_2)
rmsle(predict_R_2,train['registered'])

In [ ]:
predict_R_2_test=np.where(predict_R_2_test<0,10,predict_R_2_test)


In [ ]:
rmsle(predict_R_2,train['registered'])

In [ ]:
##############Now lets try to improve the model for casual customers###############
model_C_1=sm.OLS(train_C_y1,train_x1).fit()
print(model_C_1.summary())

In [ ]:
#H6 : casual users count has no impact on the office hours

sns.barplot(data=train,x='hour',y='casual')

In [ ]:
#the hypothesis is true
train_x1=train['hour']
train_C_y1=train['casual']
#model to predict the casual users count
model_C_1=sm.OLS(train_C_y1,train_x1).fit()
print(model_C_1.summary())

In [ ]:
#now lets see the casual users count on the days

sns.barplot(data=train,x='day',y='casual')

In [ ]:
#casual users count on the weekends is high.
sns.barplot(data=train,x='workingday',y='casual')

In [ ]:
train_x1=train[['hour','workingday']]
train_C_y1=train['casual']
#model to predict the casual users count
model_C_1=sm.OLS(train_C_y1,train_x1).fit()
print(model_C_1.summary())

In [ ]:
#R squared has improved after adding the working day variable

In [ ]:
#H7 : lets see how the casual users count changes with time
sns.barplot(data=train,x='Quarter_number',y='casual')

In [ ]:
train_x1=train[['hour','workingday','Quarter_number']]
train_C_y1=train['casual']
#model to predict the casual users count
model_C_1=sm.OLS(train_C_y1,train_x1).fit()
print(model_C_1.summary())

In [ ]:
#lets see how the temperature,humidity and windspeed is affecting the demand
sns.barplot(data=train,x='humiditybins',y='casual')

In [ ]:
sns.barplot(data=train,x='atempbins',y='casual')

In [ ]:
sns.barplot(data=train,x='tempbins',y='casual')

In [ ]:
sns.barplot(data=train,x='windspeedbins',y='casual')

In [ ]:
train_x1=train[['hour','workingday','Quarter_number','windspeedbins','atempbins','tempbins','humiditybins']]
train_C_y1=train['casual']
#model to predict the casual users count
model_C_1=sm.OLS(train_C_y1,train_x1).fit()
print(model_C_1.summary())

In [ ]:
#R squared is now increased to .639

In [ ]:
vif=pd.DataFrame()
vif['viffactor']=[variance_inflation_factor(train_x1.values,i) for i in range(train_x1.shape[1])]
vif['features']=train_x1.columns
vif

In [ ]:
#lets remove atempbins
train_x1=train[['hour','workingday','Quarter_number','windspeedbins','tempbins','humiditybins']]
train_C_y1=train['casual']
#model to predict the casual users count
model_C_1=sm.OLS(train_C_y1,train_x1).fit()
print(model_C_1.summary())

In [ ]:
#lets check how weather affect the casual customer
sns.barplot(data=train,x='weather',y='casual')

In [ ]:
#lets remove atempbins
train_x1=train[['hour','workingday','Quarter_number','windspeedbins','tempbins','humiditybins','weather_1','weather_2','weather_3','weather_4']]
train_C_y1=train['casual']
#model to predict the casual users count
model_C_1=sm.OLS(train_C_y1,train_x1).fit()
print(model_C_1.summary())

In [ ]:
vif=pd.DataFrame()
vif['viffactor']=[variance_inflation_factor(train_x1.values,i) for i in range(train_x1.shape[1])]
vif['features']=train_x1.columns
vif

In [ ]:
#lets remove atempbins
train_x1=train[['hour','workingday','Quarter_number','windspeedbins','tempbins','humiditybins','weather_1','weather_2','weather_3']]
train_C_y1=train['casual']
#model to predict the casual users count
model_C_1=sm.OLS(train_C_y1,train_x1).fit()
print(model_C_1.summary())

In [ ]:
vif=pd.DataFrame()
vif['viffactor']=[variance_inflation_factor(train_x1.values,i) for i in range(train_x1.shape[1])]
vif['features']=train_x1.columns
vif

In [ ]:
#lets remove windspeed as it is not relevant
train_x1=train[['hour','workingday','Quarter_number','tempbins','humiditybins','weather_1','weather_2','weather_3']]
train_C_y1=train['casual']
#model to predict the casual users count
model_C_1=sm.OLS(train_C_y1,train_x1).fit()
print(model_C_1.summary())

In [ ]:
sns.barplot(data=train,x='season',y='casual')

In [ ]:

train_x1=train[['hour','workingday','Quarter_number','tempbins','humiditybins','weather_1','weather_2','weather_3','season_1','season_2','season_3','season_4']]
train_C_y1=train['casual']
#model to predict the casual users count
model_C_1=sm.OLS(train_C_y1,train_x1).fit()
print(model_C_1.summary())

In [ ]:
vif=pd.DataFrame()
vif['viffactor']=[variance_inflation_factor(train_x1.values,i) for i in range(train_x1.shape[1])]
vif['features']=train_x1.columns
vif

In [ ]:

train_x1=train[['hour','workingday','Quarter_number','tempbins','humiditybins','weather_1','weather_2','weather_3','season_1','season_2','season_3']]
train_C_y1=train['casual']
#model to predict the casual users count
model_C_1=sm.OLS(train_C_y1,train_x1).fit()
print(model_C_1.summary())

In [ ]:
vif=pd.DataFrame()
vif['viffactor']=[variance_inflation_factor(train_x1.values,i) for i in range(train_x1.shape[1])]
vif['features']=train_x1.columns
vif

In [ ]:

train_x1=train[['hour','workingday','Quarter_number','tempbins','humiditybins','weather_1','weather_2','weather_3','season_2','season_3']]
train_C_y1=train['casual']
#model to predict the casual users count
model_C_1=sm.OLS(train_C_y1,train_x1).fit()
print(model_C_1.summary())

In [ ]:
vif=pd.DataFrame()
vif['viffactor']=[variance_inflation_factor(train_x1.values,i) for i in range(train_x1.shape[1])]
vif['features']=train_x1.columns
vif

In [ ]:

train_x1=train[['hour','workingday','Quarter_number','tempbins','humiditybins','weather_1','weather_2','weather_3','season_3']]
train_C_y1=train['casual']
#model to predict the casual users count
model_C_1=sm.OLS(train_C_y1,train_x1).fit()
print(model_C_1.summary())

In [ ]:
test_x1=test[['hour','workingday','Quarter_number','tempbins','humiditybins','weather_1','weather_2','weather_3','season_3']]

In [ ]:
#predict the Registered and casual users
predict_C_1=model_C_1.predict(train_x1)

#visualize predict_count_1 and actual count
plt.Figure(figsize=(5,5))
plt.scatter(predict_C_1,train['casual'])
plt.plot([0,500],[0,500],color='red')
plt.xlim=(-100,500)
plt.ylim=(-100,500)
plt.xlabel('prediction')
plt.ylabel('train_y')
plt.title('Linear Regression Model')

In [ ]:
predict_C_2=np.where(predict_C_1<0,10,predict_C_1) #predicted casual users count

In [ ]:
train_x1.info()

In [ ]:
test_x1.info()

In [ ]:
predict_C_1_test=model_C_1.predict(test_x1) #predicting the the casual users count on test data

In [ ]:
predict_C_2_test=np.where(predict_C_1_test<0,10,predict_C_1_test) #convert the -ve numbers to +ve


In [ ]:
predict_C_2_test

In [ ]:
rmsle(predict_C_2,train['casual']) #RMSLE for the predicted casual users count on the train.

In [ ]:
rmsle(predict_R_2,train['registered']) #RMSLE for the predicted registered users count on the train

In [ ]:
final_predict_count=predict_C_2+predict_R_2 #final count for the train data set

In [ ]:
final_predict_count_test=predict_C_2_test+predict_R_2_test #final count for the test data set

In [ ]:
final_predict_count_test

In [ ]:
rmsle(final_predict_count,train['count'])

In [ ]:
final_predict_count_test.shape

In [ ]:

test=pd.read_csv("../input/test.csv")
test.info()

In [ ]:
d={'datetime':test['datetime'],'count':final_predict_count_test}

In [ ]:
ans=pd.DataFrame(d)

In [ ]:
ans.to_csv('answer.csv',index=False) # saving to a csv file for predictions on kaggle.